In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint

import numpy as np
from skimage import data
from skimage.transform import resize

import matplotlib.pyplot as plt

import torch

from src.reader import KineticDataset

from src.vqgan import ViTVQGAN
from src.my_model import MaskCode
from src.trainer import AttentionMaskModeling


In [3]:
path = "/mnt/e/kinetics-dataset/k400"
split = "train"

ds = KineticDataset(
    path, split,
    n_frames=16,
)
ds_loader = DataLoader(
    ds, 1, True, 
)

99/246534

In [4]:
for batch in ds_loader:
    code, _ = batch
    break

B, T, HW = code.shape

In [11]:
# lightning_model = AttentionMaskModeling.load_from_checkpoint(
#     "./trained/semcom-epoch=14-train_acc=0.96.ckpt"
# )#.to(torch.float16)

lightning_model = AttentionMaskModeling(
    model=MaskCode(
		window_size=(3, 3),
		length=32, height=32, width=32,
		depth=4, heads=8, dim=128, embed_dim=32,
		n_codes=8192,
	), 
    top_p=0.95
)
lightning_model.cuda()
_batch = batch[0].cuda(), None
# lightning_model.training_step(_batch, 0)

/home/leonard/anaconda3/envs/semcom/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:210: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.


In [19]:
def _test(model, data, device="cpu", dtype=torch.float32):
    # with torch.autocast(device_type=device, dtype=dtype), torch.no_grad():
    model.get_mask_from_logits(data)

%timeit _test(lightning_model, _batch[0], "cuda", torch.float16)

25.5 ms ± 15.3 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
def _test_2(model, img):
    with torch.autocast(device_type='cuda', dtype=torch.float16):
        model(img)

vitvq = ViTVQGAN.get_vit_vqgan_base()
vitvq.init_from_ckpt("./checkpoint/imagenet_vitvq_base.ckpt")
vitvq.eval()
vitvq.cuda()

data = torch.rand((1, 3, 256, 256)).cuda()

%timeit _test_2(vitvq, data)

In [20]:
def _test(model, data, device="cpu", dtype=torch.float32):
    model.reset_cache()
    # with torch.autocast(device_type=device, dtype=dtype):
    for d in data:
        model.get_mask_from_logits(d[None, ...], use_cache=True)

         
%timeit _test(lightning_model, _batch[0], "cuda", torch.float16)

25.5 ms ± 149 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
frame[None, ...].shape

In [16]:
lightning_model.reset_cache()
with torch.autocast(device_type="cuda", dtype=torch.float32), torch.no_grad():
    for i, frame in enumerate(_batch[0]):
        print("step:", i)
        lightning_model.get_mask_from_logits(frame[None, ...], use_cache=True)

        # break

step: 0
------------------------------
kv torch.Size([16384, 8, 10, 16]) torch.Size([16384, 8, 10, 16])
kv torch.Size([16384, 8, 10, 16]) torch.Size([16384, 8, 10, 16])
------------------------------
kv torch.Size([1024, 8, 17, 16]) torch.Size([1024, 8, 17, 16])
kv torch.Size([1024, 8, 17, 16]) torch.Size([1024, 8, 17, 16])
------------------------------
kv torch.Size([16384, 8, 10, 16]) torch.Size([16384, 8, 10, 16])
kv torch.Size([16384, 8, 10, 16]) torch.Size([16384, 8, 10, 16])
------------------------------
kv torch.Size([1024, 8, 17, 16]) torch.Size([1024, 8, 17, 16])
kv torch.Size([1024, 8, 17, 16]) torch.Size([1024, 8, 17, 16])
------------------------------
kv torch.Size([16384, 8, 10, 16]) torch.Size([16384, 8, 10, 16])
kv torch.Size([16384, 8, 10, 16]) torch.Size([16384, 8, 10, 16])
------------------------------
kv torch.Size([1024, 8, 17, 16]) torch.Size([1024, 8, 17, 16])
kv torch.Size([1024, 8, 17, 16]) torch.Size([1024, 8, 17, 16])
------------------------------
kv torc